In [1]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn import (metrics, linear_model, preprocessing)
from sklearn.model_selection import train_test_split 
import xgboost as xgb 
from matplotlib import pyplot
# feature selection    make sure to update xgboost package
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, RepeatedKFold, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, KFold
from scipy.stats import uniform,randint
from xgboost import plot_importance

In [2]:
# Function importing Dataset 
def read_data(path): 
    df = pd.read_csv(path, na_values='NA')
    # Printing the dataswet shape 
    print ("features:") 
    print (list(df.columns)) 
    print ("row and column number") 
    print (df.shape) 
    print ("data / feature types:") 
    print (df.dtypes) 
    df_num = df.select_dtypes(include='number')
    df_cat = df.select_dtypes(include='object')
    print ("missing values:") 
    print (df.isnull().sum()) 
    return [df, df_num, df_cat] 


In [5]:
############## multi-classification problem:  'mlogloss' or "merror"#################################
path = './../data/trainforestcover.csv'
dflist = read_data(path)

df = dflist[0]
predictor = list(df.columns)

 # remove constant columns
remove = []
for col in df.columns:
    if df[col].std() == 0:
        remove.append(col)

df = df.drop(remove, axis=1)

# remove duplicated columns
remove = []
c = df.columns
for i in range(len(c)-1):
    v = df[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v, df[c[j]].values):
            remove.append(c[j])       
        
df = df.drop(remove, axis=1)

set(df['Cover_Type']) # 7 types

y = df['Cover_Type'] - 1

set(y)

X = df.drop(['Id','Cover_Type'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.25, random_state = 19) 


dtrain = xgb.DMatrix(data=X_train,
                     feature_names=X_train.columns,
                     label=y_train)

dvalid = xgb.DMatrix(data=X_valid,
                     feature_names=X_valid.columns,
                     label=y_valid)

params = {'learning_rate' : 0.03,
    'max_depth' : 7,
    'objective': 'multi:softmax',
    'min_child_weight' : 5, 
    'gamma' : 0.05, 
    'subsample': 0.8, 
    'colsample_bytree': 0.8,
    'seed' : 12,
    'num_class' : 7,
    'eval_metric' : 'mlogloss',  # can also be 'merror'
    'tree_method': 'gpu_hist',
    'sampling_method': 'gradient_based'
}


features:
['Id', 'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Cover_Type']
row and column number
(15120, 56)
data / feature types:
Id             

In [6]:

eval_cv = xgb.cv(params, dtrain,
  num_boost_round=1000, nfold=3, shuffle=True, verbose_eval=5, early_stopping_rounds=50)

eval_cv.columns
eval_cv.head()

[0]	train-mlogloss:1.88905+0.00034	test-mlogloss:1.89187+0.00040
[5]	train-mlogloss:1.66084+0.00105	test-mlogloss:1.67492+0.00108
[10]	train-mlogloss:1.48915+0.00138	test-mlogloss:1.51439+0.00185
[15]	train-mlogloss:1.34458+0.00236	test-mlogloss:1.37918+0.00170
[20]	train-mlogloss:1.22867+0.00339	test-mlogloss:1.27182+0.00177
[25]	train-mlogloss:1.12992+0.00454	test-mlogloss:1.18009+0.00191
[30]	train-mlogloss:1.04608+0.00480	test-mlogloss:1.10253+0.00200
[35]	train-mlogloss:0.97293+0.00530	test-mlogloss:1.03514+0.00186
[40]	train-mlogloss:0.91081+0.00565	test-mlogloss:0.97793+0.00171
[45]	train-mlogloss:0.85748+0.00565	test-mlogloss:0.92906+0.00163
[50]	train-mlogloss:0.80964+0.00550	test-mlogloss:0.88561+0.00161
[55]	train-mlogloss:0.76745+0.00525	test-mlogloss:0.84761+0.00182
[60]	train-mlogloss:0.72858+0.00557	test-mlogloss:0.81260+0.00186
[65]	train-mlogloss:0.69508+0.00547	test-mlogloss:0.78284+0.00205
[70]	train-mlogloss:0.66573+0.00554	test-mlogloss:0.75661+0.00232
[75]	train-m

,train-mlogloss-mean,train-mlogloss-std,test-mlogloss-mean,test-mlogloss-std
0,1.889045,0.000342,1.891865,0.000399
1,1.837485,0.000283,1.842658,0.000858
2,1.787178,0.000644,1.794584,0.001025
3,1.740654,0.000607,1.750274,0.000851
4,1.699216,0.000840,1.710914,0.001003


In [7]:
# check log loss: -log P(yt|yp) = -(yt log(yp) + (1 - yt) log(1 - yp))
nround = eval_cv['test-mlogloss-mean'].idxmin()
# final model
model = xgb.train(params, dtrain, num_boost_round=nround)

#### usando otra métrica de evaluación y API de scikit-learn

In [8]:
# sklean XGBoostClassifier, we use eval_metric="merror"
y= df['Cover_Type'].values
y = [x-1 for x in y]
X = df.drop(['Id','Cover_Type'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.25, random_state = 19) 


# classifier
clf = xgb.XGBClassifier(max_depth=7, 
   n_estimators=2500, learning_rate=0.02, nthread=4, subsample=0.9, 
   colsample_bytree=0.8, seed=9, tree_method='gpu_hist', sampling_method='gradient_based')

# fitting
clf.fit(X_train, y_train, early_stopping_rounds=50, eval_metric="merror", eval_set=[(X_valid, y_valid)])

[0]	validation_0-merror:0.28889


e:\udemy\xgboost\xgboost_env\Lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
e:\udemy\xgboost\xgboost_env\Lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[1]	validation_0-merror:0.24921
[2]	validation_0-merror:0.23968
[3]	validation_0-merror:0.23889
[4]	validation_0-merror:0.23466
[5]	validation_0-merror:0.22884
[6]	validation_0-merror:0.22910
[7]	validation_0-merror:0.23069
[8]	validation_0-merror:0.22698
[9]	validation_0-merror:0.22698
[10]	validation_0-merror:0.22778
[11]	validation_0-merror:0.22646
[12]	validation_0-merror:0.22937
[13]	validation_0-merror:0.22963
[14]	validation_0-merror:0.22725
[15]	validation_0-merror:0.22619
[16]	validation_0-merror:0.22593
[17]	validation_0-merror:0.22513
[18]	validation_0-merror:0.22460
[19]	validation_0-merror:0.22196
[20]	validation_0-merror:0.22143
[21]	validation_0-merror:0.22037
[22]	validation_0-merror:0.22116
[23]	validation_0-merror:0.22196
[24]	validation_0-merror:0.22037
[25]	validation_0-merror:0.22249
[26]	validation_0-merror:0.22249
[27]	validation_0-merror:0.22116
[28]	validation_0-merror:0.22169
[29]	validation_0-merror:0.22143
[30]	validation_0-merror:0.22116
[31]	validation_0-m

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=0,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.02, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=7, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=2500, n_jobs=4, nthread=4,
              num_parallel_tree=1, objective='multi:softprob', ...)

: 